# OSVOS Parent Model Results

## Imports

In [1]:
import os

import scipy.misc as sm
import numpy as np
import torch
from torch.utils.data import DataLoader

import OSVOS_PyTorch.networks.vgg_osvos as vo
from OSVOS_PyTorch.dataloaders import davis_2016 as db
from OSVOS_PyTorch.dataloaders import custom_transforms as tr
from OSVOS_PyTorch.mypath import Path

import src.config as cfg

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
db_root_dir = Path.db_root_dir()
save_dir = cfg.PARENT_MODEL_RESULTS_PATH

if not os.path.exists(save_dir):
    os.makedirs(os.path.join(save_dir))

In [4]:
gpu_id = 0
device = torch.device("cuda:"+str(gpu_id) if torch.cuda.is_available() else "cpu")

In [5]:
net = vo.OSVOS(pretrained=0)
net.load_state_dict(torch.load(cfg.PARENT_MODEL_PATH, map_location=lambda storage, loc: storage))
net.to(device)

Constructing OSVOS architecture..
Initializing weights..


OSVOS(
  (upscale): ModuleList(
    (0): ConvTranspose2d(16, 16, kernel_size=(4, 4), stride=(2, 2), bias=False)
    (1): ConvTranspose2d(16, 16, kernel_size=(8, 8), stride=(4, 4), bias=False)
    (2): ConvTranspose2d(16, 16, kernel_size=(16, 16), stride=(8, 8), bias=False)
    (3): ConvTranspose2d(16, 16, kernel_size=(32, 32), stride=(16, 16), bias=False)
  )
  (upscale_): ModuleList(
    (0): ConvTranspose2d(1, 1, kernel_size=(4, 4), stride=(2, 2), bias=False)
    (1): ConvTranspose2d(1, 1, kernel_size=(8, 8), stride=(4, 4), bias=False)
    (2): ConvTranspose2d(1, 1, kernel_size=(16, 16), stride=(8, 8), bias=False)
    (3): ConvTranspose2d(1, 1, kernel_size=(32, 32), stride=(16, 16), bias=False)
  )
  (stages): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace)
    )
    (1): Sequential(
      (0): MaxP

In [9]:
for seq_name in cfg.VAL_SEQUENCES[1:]:
    db_test = db.DAVIS2016(train=False, db_root_dir=db_root_dir,
                           transform=tr.ToTensor(), 
                           seq_name=seq_name)
    testloader = DataLoader(db_test, batch_size=1, shuffle=False, num_workers=1)

    num_img_ts = len(testloader)

    save_dir_res = os.path.join(save_dir, seq_name)
    if not os.path.exists(save_dir_res):
        os.makedirs(save_dir_res)

    print('Testing Network')
    with torch.no_grad():  
        # Main Testing Loop
        for ii, sample_batched in enumerate(testloader):

            img, gt, fname = sample_batched['image'], sample_batched['gt'], sample_batched['fname']

            # Forward of the mini-batch
            inputs, gts = img.to(device), gt.to(device)

            outputs = net.forward(inputs)

            for jj in range(int(inputs.size()[0])):
                pred = np.transpose(outputs[-1].cpu().data.numpy()[jj, :, :, :], (1, 2, 0))
                pred = 1 / (1 + np.exp(-pred))
                pred = np.squeeze(pred)

                # Save the result, attention to the index jj
                sm.imsave(os.path.join(save_dir_res, os.path.basename(fname[jj]) + '.png'), pred)

Done initializing val_seqs Dataset
Testing Network


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Done initializing val_seqs Dataset
Testing Network
Done initializing val_seqs Dataset
Testing Network
Done initializing val_seqs Dataset
Testing Network
Done initializing val_seqs Dataset
Testing Network
Done initializing val_seqs Dataset
Testing Network
Done initializing val_seqs Dataset
Testing Network
Done initializing val_seqs Dataset
Testing Network
Done initializing val_seqs Dataset
Testing Network
Done initializing val_seqs Dataset
Testing Network
Done initializing val_seqs Dataset
Testing Network
Done initializing val_seqs Dataset
Testing Network
Done initializing val_seqs Dataset
Testing Network
Done initializing val_seqs Dataset
Testing Network
Done initializing val_seqs Dataset
Testing Network
Done initializing val_seqs Dataset
Testing Network
Done initializing val_seqs Dataset
Testing Network
Done initializing val_seqs Dataset
Testing Network
Done initializing val_seqs Dataset
Testing Network
